<a href="https://colab.research.google.com/github/HongFFF/OPTI-556/blob/main/556_Term_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OPTI 556 Term Project: A Neural Network that Improves the Image Quality of a List-Mode CT/SPECT Imaging System
### Yifan Hong

In the term project, I plan to use convolutional neural networks to improve the reconstructed image quality for a CT imaging system that uses list-mode data. But after some investigation on the math and code, I found that the SPECT system is more practical as the computation time in the forward model is much smaller than the CT system.

List mode data has a lot of advantages, like better reconstruction accuracy, less storage needed, and more adaptability in the computation. However, the major problem is that the reconstruction needs much higher computation power than the traditional dataset. The complexity of reconstruction algorithm is linear depend on the number of photons, which is normally a large number in one measurement. So in this project I plan to use a convolutional neural network to improve the image quality with limit number of photons.



### Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

from google.colab import files

from skimage.restoration import denoise_tv_chambolle

### Functions

In [ ]:
def genData():
  ### Forward
  ptheta = np.zeros((Nphoton,2))
  Nellipse = np.random.poisson(lam = Nellipse_mean)+1
  A = np.zeros(Nellipse)
  B = np.zeros(Nellipse)
  beta = np.zeros(Nellipse)
  xs = np.zeros(Nellipse)
  ys = np.zeros(Nellipse)
  area = np.zeros(Nellipse)
  Cumarea = np.zeros(Nellipse)
  for i in range(Nellipse):
    A[i], B[i], beta[i], xs[i], ys[i] = gen_ellipse(A_mean, B_mean, Lxs, Lys)
    area[i] = A[i]*B[i]
    Cumarea[i] = area[i]
    if i>0:
      Cumarea[i] = Cumarea[i] + Cumarea[i-1]
  Cumarea = Cumarea/np.sum(area)
  for i in range(Nphoton):
    Nsource = Nellipse - sum(np.random.random()<Cumarea)
    ptheta[i:i+1,:] = gen_photon_ptheta(A[Nsource], B[Nsource], beta[Nsource], xs[Nsource], ys[Nsource])
  ### Golden Truth
  obj_true = np.zeros(X.shape)
  for i in range(Nellipse):
    obj_true = obj_true+plot_ellipse(A[i],B[i],beta[i],xs[i],ys[i],X,Y)
  ### FBP
  obj_recon = np.zeros(X.shape)
  for i in range(Nphoton):
    obj_recon = obj_recon+backproj(ptheta[i,0],ptheta[i,1],epsilon, X, Y)
  ### Normalization
  obj_true = obj_true/np.max(obj_true)
  obj_recon = obj_recon/np.max(obj_recon)
  return obj_recon, obj_true

def genDatawithTV():
  ### Forward
  ptheta = np.zeros((Nphoton,2))
  Nellipse = np.random.poisson(lam = Nellipse_mean)+1
  A = np.zeros(Nellipse)
  B = np.zeros(Nellipse)
  beta = np.zeros(Nellipse)
  xs = np.zeros(Nellipse)
  ys = np.zeros(Nellipse)
  area = np.zeros(Nellipse)
  Cumarea = np.zeros(Nellipse)
  for i in range(Nellipse):
    A[i], B[i], beta[i], xs[i], ys[i] = gen_ellipse(A_mean, B_mean, Lxs, Lys)
    area[i] = A[i]*B[i]
    Cumarea[i] = area[i]
    if i>0:
      Cumarea[i] = Cumarea[i] + Cumarea[i-1]
  Cumarea = Cumarea/np.sum(area)
  for i in range(Nphoton):
    Nsource = Nellipse - sum(np.random.random()<Cumarea)
    ptheta[i:i+1,:] = gen_photon_ptheta(A[Nsource], B[Nsource], beta[Nsource], xs[Nsource], ys[Nsource])
  ### Golden Truth
  obj_true = np.zeros(X.shape)
  for i in range(Nellipse):
    obj_true = obj_true+plot_ellipse(A[i],B[i],beta[i],xs[i],ys[i],X,Y)
  ### FBP
  obj_recon = np.zeros(X.shape)
  for i in range(Nphoton):
    obj_recon = obj_recon+backproj(ptheta[i,0],ptheta[i,1],epsilon, X, Y)  
  obj_recon = obj_recon/np.max(obj_recon)
  obj_recon_orig = obj_recon
  obj_recon = denoise_tv_chambolle(obj_recon, weight = 0.5)
  ### Normalization
  obj_true = obj_true/np.max(obj_true)
  obj_recon = obj_recon/np.max(obj_recon)
  return obj_recon, obj_true, obj_recon_orig

In [ ]:
def gen_ellipse(A_mean, B_mean, Lxs, Lys):
  A = abs(np.random.randn()*A_mean+A_mean)
  B = abs(np.random.randn()*B_mean+B_mean)
  beta = np.random.random()*np.pi
  xs = np.random.random()*Lxs-Lxs/2
  ys = np.random.random()*Lys-Lys/2
  return A, B, beta, xs, ys

def plot_ellipse(A,B,beta,xs,ys,X,Y):
  obj = np.zeros(X.shape)
  obj = ((X+xs)*np.cos(beta)-(Y+ys)*np.sin(beta))**2/A**2+((X+xs)*np.sin(beta)+(Y+ys)*np.cos(beta))**2/B**2
  obj = obj<=1
  obj = obj.astype(float)
  return obj

def gen_photon_ptheta(A,B,beta,xs,ys):
  theta = np.random.random()*2*np.pi
  p =  np.sqrt(np.random.random())
  x,y = ptheta2xy(p,theta)
  x = x*A
  y = y*B
  p, theta = xy2ptheta(x,y)
  theta = theta-beta
  x, y = ptheta2xy(p, theta)
  x = x+xs
  y = y+ys
  theta = np.random.random()*np.pi
  p = -x*np.cos(theta)-y*np.sin(theta)
  return [p, theta]

def backproj(p, theta, epsilon, X, Y):
  dist = np.abs(p-X*np.cos(theta)-Y*np.sin(theta))
  dist2 = dist**2
  return -(2*dist2)/(dist2+epsilon**2)**2+1/(dist2+epsilon**2)

#def backproj(p, theta, epsilon, X, Y):
#  dx = X[0,1]-X[0,0]
#  dy = Y[1,0]-Y[0,0]
#  e2 = epsilon**2
#  Z1 = X*np.cos(theta)+(Y+dy)*np.sin(theta)-p
#  Z2 = X*np.cos(theta)+Y*np.sin(theta)-p
#  Z3 = (X+dx)*np.cos(theta)+Y*np.sin(theta)-p
#  Z4 = (X+dx)*np.cos(theta)+(Y+dy)*np.sin(theta)-p
#  Int = np.log(Z1**2+e2)-np.log(Z2**2+e2)+np.log(Z3**2+e2)-np.log(Z4**2+e2)
#  const = -1/(2*np.cos(theta)*np.sin(theta))
#  return Int*const



#### Less Major Functions

In [ ]:
def ptheta2xy(p,theta):
  x = p*np.cos(theta)
  y = p*np.sin(theta)
  return x, y

def xy2ptheta(x,y):
  if y>0:
    sign = 1
  else:
    sign = -1
  p = np.sqrt(x**2+y**2)*sign
  x = x*sign
  theta = math.acos(x/np.sqrt(x**2+y**2))
  return p, theta

def calc_prob(A,B,beta,alpha,xs,ys):
  factor1 = (1/A**2+1/B**2)
  termA = factor1**2*(xs*math.cos(alpha)*math.cos(beta)**2+ys*math.sin(alpha)*math.cos(beta)**2-math.cos(alpha)*math.cos(beta)-math.sin(alpha)*math.sin(beta))*2
  termB = factor1*(math.cos(alpha)**2*math.cos(beta)**2+math.sin(alpha)**2*math.sin(beta)**2)
  termC = factor1*(xs**2*math.cos(beta)+ys**2*math.sin(beta)-2*xs*math.cos(beta)-2*ys*math.sin(beta))-1
  ALongOne = A-B*C
  if ALongOne>0:
    Lpath = 2*np.sqrt(ALongOne)/factor1/(math.cos(alpha)**2*math.cos(beta)**2-math.sin(alpha)**2*math.sin(beta)**2)
  else:
    Lpath = 0
  return np.exp(-mu*Lpath)


### Parameters
Most of the parameters are shown as below. The mean number of ellipse is chosen as 5, and the true number is distributed as a Poisson distribution. The long/short axes use 0.2 as the mean, where the computational window is 2 by 2 with 128 pixels on each side. The origin of the ellipses located at the center in a 1 by 1 area. The orientation is uniformly distributed in pi angle. 


In [ ]:
Lx = 2
Ly = 2
Lp = 3
Lxs = 1
Lys = 1

Nx = 128
Ny = 128
Lsource = 1
Nphoton= 10000

Nellipse_mean = 5
A_mean = 0.2
B_mean = 0.2
mu_mean = 4

In [ ]:
x = np.linspace(-Lx/2,Lx/2,Nx+1)
x = x[0:Nx]
y = np.linspace(-Ly/2,Ly/2,Ny+1)
y = y[0:Ny]
dx = x[2]-x[1]
dy = y[2]-y[1]
X, Y = np.meshgrid(x,y)

epsilon = 1*np.sqrt(dx*dy)

### Some test on the Forward and Backward model
In this section, I briefly tested on the results of the object generator. And then the random object was used to show the validation of the forward and backward model of the SPECT system. 

#### Forward Model

In [ ]:
ptheta = np.zeros((Nphoton,2))

Nellipse = np.random.poisson(lam = Nellipse_mean)+1
#
#Nellipse = 1
#
A = np.zeros(Nellipse)
B = np.zeros(Nellipse)
beta = np.zeros(Nellipse)
xs = np.zeros(Nellipse)
ys = np.zeros(Nellipse)
area = np.zeros(Nellipse)
Cumarea = np.zeros(Nellipse)

for i in range(Nellipse):
  A[i], B[i], beta[i], xs[i], ys[i] = gen_ellipse(A_mean, B_mean, Lxs, Lys)
  area[i] = A[i]*B[i]
  Cumarea[i] = area[i]
  if i>0:
    Cumarea[i] = Cumarea[i] + Cumarea[i-1]
Cumarea = Cumarea/np.sum(area)

#
#beta = np.zeros(Nellipse)
#xs = np.zeros(Nellipse)
#ys = np.zeros(Nellipse)
#B = A
#

for i in range(Nphoton):
 Nsource = Nellipse - sum(np.random.random()<Cumarea)
 ptheta[i:i+1,:] = gen_photon_ptheta(A[Nsource], B[Nsource], beta[Nsource], xs[Nsource], ys[Nsource])

#### Print the object
A random object from the ellipses generator

In [ ]:
obj = np.zeros(X.shape)
for i in range(Nellipse):
  obj = obj+plot_ellipse(A[i],B[i],beta[i],xs[i],ys[i],X,Y)
plt.figure()
plt.imshow(obj)

#### FBP
The estimated object with 10k photons with the list mode data

In [ ]:
obj = np.zeros(X.shape)
for i in range(Nphoton):
  obj = obj+backproj(ptheta[i,0],ptheta[i,1],epsilon, X, Y)
plt.figure()
obj = obj/np.max(obj)
obj = denoise_tv_chambolle(obj, weight = 0.2)
plt.imshow(obj)

### Generate Dataset without Total Variation regularization

Here is the first result, which the inputs to the neural network are directly calculated from the FBP. 1000 object are used as the training dataset and 100 objects are used as the validation dataset.  

#### Preparing Data

In [ ]:
Ntrain = 1000
Ntest = 100

xtrain = np.zeros((Ntrain,Nx,Ny))
ytrain = np.zeros((Ntrain,Nx,Ny))
xtest = np.zeros((Ntest,Nx,Ny))
ytest = np.zeros((Ntest,Nx,Ny))

for i in range(Ntest):
  xtest[i:i+1,:,:], ytest[i:i+1,:,:] = genData()
for i in range(Ntrain):
  xtrain[i:i+1,:,:], ytrain[i:i+1,:,:] = genData()

xtrain = np.reshape(xtrain, (len(xtrain), Nx, Ny, 1))
ytrain = np.reshape(ytrain, (len(ytrain), Nx, Ny, 1))
xtest = np.reshape(xtest, (len(xtest), Nx, Ny, 1))
ytest = np.reshape(ytest, (len(ytest), Nx, Ny, 1))

#### Or Load Data From Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

trainingData = np.load('drive/MyDrive/Colab Notebooks/DataSave/trainingData.npz')
testData = np.load('drive/MyDrive/Colab Notebooks/DataSave/testData.npz')

xtrain=trainingData['arr_0']
ytrain=trainingData['arr_1']

xtest=testData['arr_0']
ytest=testData['arr_1']

#### CNN1

In [ ]:
inputs=keras.Input(shape=(Nx, Ny, 1))

# Encoder
x = layers.Conv2D(64, (3, 3), strides=1, activation="relu", padding="same")(inputs)
x1 = layers.Conv2D(128, (8, 8), strides=2, activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x1)
x = layers.Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)


# Decoder
x2 = layers.Conv2DTranspose(128, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Add()([x1,x2])
x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(16, (4, 4), strides=1, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(8, (8, 8), activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(1, (3, 3), activation="linear", padding="same")(x)

SPECT_denoise = keras.Model(inputs, x)
SPECT_denoise.compile(optimizer='adam', loss='mean_squared_error')
SPECT_denoise.summary()

#### CNN2

In [ ]:
inputs=keras.Input(shape=(Nx, Ny, 1))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = layers.Dropout(0.2)(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.Dropout(0.2)(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# At this point the representation is (7, 7, 32)

x = layers.Conv2DTranspose(32, (3, 3), activation='relu',strides= 2, padding='same')(encoded)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu',strides= 2, padding='same')(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

SPECT_denoise = keras.Model(inputs, decoded)
SPECT_denoise.compile(optimizer='adam', loss='binary_crossentropy')
SPECT_denoise.summary()

In [ ]:
SPECT_denoise.fit(
    x=xtrain,
    y=ytrain,
    epochs=20,
    batch_size=100,
    shuffle=True,
    validation_data=(xtest, ytest),
)

##### Save Data to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
np.savez('drive/MyDrive/Colab Notebooks/DataSave/trainingData',xtrain,ytrain)
np.savez('drive/MyDrive/Colab Notebooks/DataSave/testData',xtest,ytest)
SPECT_denoise.save('drive/MyDrive/Colab Notebooks/DataSave/SPECT_denoise') 


SPECT_denoise.save("SPECT_denoise")
!tar -czvf SPECT_denoise.tar.gz SPECT_denoise/

#files.download('testData.npz')
#files.download('trainingData.npz')
#files.download('SPECT_denoise.tar.gz')

##### Load Neural Network

In [ ]:
SPECT_denoise = keras.models.load_model('drive/MyDrive/Colab Notebooks/DataSave/SPECT_denoise')

### Results Plot without Total Variation regularization
Results from CNN: the first row are the golden truth, the second row are the result by FBP, and the last row are the estimation from the CNN

From the result, we can observe that the neural network has a significant improvement on the reconstruction.

In [ ]:
n = 5
plt.figure(figsize=(15, 10))
for i in range(1, n + 1):
    ax = plt.subplot(3, n, i)
    plt.imshow(ytest[3*i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(3, n, n+i)
    plt.imshow(xtest[3*i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(3, n, 2*n+i)
    plt.imshow(SPECT_denoise.predict(xtest[3*i:3*i+1,:,:,:])[0,:,:,0])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

### Convolutional Neural Network with Total Variation Regularization
Thanks to the advice received during the presentation, I added a total variation regularization before the input of the neural network. The weight is tuned based on the mse on the validation dataset. The results show that after the TV regularization, even with the same setup for the neural network, the estimation on the objects is better than the case without the TV regularization.

#### Preparing Data

In [ ]:
Ntrain = 1000
Ntest = 100

xtrain = np.zeros((Ntrain,Nx,Ny))
ytrain = np.zeros((Ntrain,Nx,Ny))
xtest = np.zeros((Ntest,Nx,Ny))
ytest = np.zeros((Ntest,Nx,Ny))

for i in range(Ntest):
  xtest[i:i+1,:,:], ytest[i:i+1,:,:] = genDatawithTV()
for i in range(Ntrain):
  xtrain[i:i+1,:,:], ytrain[i:i+1,:,:] = genDatawithTV()

xtrain = np.reshape(xtrain, (len(xtrain), Nx, Ny, 1))
ytrain = np.reshape(ytrain, (len(ytrain), Nx, Ny, 1))
xtest = np.reshape(xtest, (len(xtest), Nx, Ny, 1))
ytest = np.reshape(ytest, (len(ytest), Nx, Ny, 1))

#### Or Load Data From Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

trainingData = np.load('drive/MyDrive/Colab Notebooks/DataSave/trainingDatawithTV2.npz')
testData = np.load('drive/MyDrive/Colab Notebooks/DataSave/testDatawithTV2.npz')

xtrain=trainingData['arr_0']
ytrain=trainingData['arr_1']

xtest=testData['arr_0']
ytest=testData['arr_1']

#### CNN1

In [ ]:
inputs=keras.Input(shape=(Nx, Ny, 1))

# Encoder
x = layers.Conv2D(64, (3, 3), strides=1, activation="relu", padding="same")(inputs)
x1 = layers.Conv2D(128, (8, 8), strides=2, activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x1)
x = layers.Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)


# Decoder
x2 = layers.Conv2DTranspose(128, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Add()([x1,x2])
x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(16, (4, 4), strides=1, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(8, (8, 8), activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(1, (3, 3), activation="linear", padding="same")(x)

SPECT_denoise = keras.Model(inputs, x)
SPECT_denoise.compile(optimizer='adam', loss='mean_squared_error')
SPECT_denoise.summary()

#### CNN2

In [ ]:
inputs=keras.Input(shape=(Nx, Ny, 1))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = layers.Dropout(0.2)(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.Dropout(0.2)(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# At this point the representation is (7, 7, 32)

x = layers.Conv2DTranspose(32, (3, 3), activation='relu',strides= 2, padding='same')(encoded)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu',strides= 2, padding='same')(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

SPECT_denoise = keras.Model(inputs, decoded)
SPECT_denoise.compile(optimizer='adam', loss='binary_crossentropy')
SPECT_denoise.summary()

In [ ]:
SPECT_denoise.fit(
    x=xtrain,
    y=ytrain,
    epochs=20,
    batch_size=100,
    shuffle=True,
    validation_data=(xtest, ytest),
)

#### Save Data to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
np.savez('drive/MyDrive/Colab Notebooks/DataSave/trainingDatawithTV2',xtrain,ytrain)
np.savez('drive/MyDrive/Colab Notebooks/DataSave/testDatawithTV2',xtest,ytest)
SPECT_denoise.save('drive/MyDrive/Colab Notebooks/DataSave/SPECT_denoise') 


SPECT_denoise.save("SPECT_denoise")
!tar -czvf SPECT_denoise.tar.gz SPECT_denoise/

#files.download('testData.npz')
#files.download('trainingData.npz')
#files.download('SPECT_denoise.tar.gz')

### Results Plot with Total Variation Regularization
Results from CNN: the first row are the golden truth, 
the second row are the result by FBP, the third row are the outputs of TV regularization, and the last row are the estimation from the CNN

From the result we can observe that the TV regularization can already decrease the artifact by a lot, and the CNN will further increase the sharpness and clear the background. It might be hard to observe the difference between the case with and without the TV regularization. But from the MSE during the training, with the same neural network, having the TV regularization can decrease the MSE by a little. 

I am still trying to increase the size of the training data, but due to the limitation on resource, it might be infeasible to do it on the Colab. I will try to implement it on other platform. 

In [ ]:
n = 5
plt.figure(figsize=(15, 10))
for i in range(1, n + 1):
    ax = plt.subplot(4, n, i)
    plt.imshow(ytest[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(4, n, n+i)
    plt.imshow(xtest_orig[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(4, n, 2*n+i)
    plt.imshow(xtest[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(4, n, 3*n+i)
    plt.imshow(SPECT_denoise.predict(xtest[i:i+1,:,:,:])[0,:,:,0])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

### Some Plots for the Presentation

#### Generate Test Data

In [ ]:
Ntest = 10

xtest = np.zeros((Ntest,Nx,Ny))
ytest = np.zeros((Ntest,Nx,Ny))
xtest_orig = np.zeros((Ntest,Nx,Ny))

for i in range(Ntest):
  xtest[i:i+1,:,:], ytest[i:i+1,:,:], xtest_orig[i:i+1,:,:]  = genDatawithTV()

xtest = np.reshape(xtest, (len(xtest), Nx, Ny, 1))
ytest = np.reshape(ytest, (len(ytest), Nx, Ny, 1))
xtest_orig = np.reshape(xtest_orig, (len(xtest_orig), Nx, Ny, 1))

#### True Objects

In [ ]:
n = 5
plt.figure(figsize=(15, 5))
for i in range(1, n + 1):
    ax = plt.subplot(1, n, i)
    plt.imshow(ytest[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

#### Estimated Objects from FBP

In [ ]:
n = 5
plt.figure(figsize=(15, 5))
for i in range(1, n + 1):
    ax = plt.subplot(1, n, i)
    plt.imshow(xtest_orig[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

#### Estimated Objects after TV Regularization

In [ ]:
n = 5
plt.figure(figsize=(15, 5))
for i in range(1, n + 1):
    ax = plt.subplot(1, n, i)
    plt.imshow(xtest[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

#### Results from the Neural Network

In [ ]:
n = 5
plt.figure(figsize=(15, 10))
for i in range(1, n + 1):
    ax = plt.subplot(4, n, i)
    plt.imshow(ytest[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(4, n, n+i)
    plt.imshow(xtest_orig[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(4, n, 2*n+i)
    plt.imshow(xtest[i].reshape(Nx, Ny))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(4, n, 3*n+i)
    plt.imshow(SPECT_denoise.predict(xtest[i:i+1,:,:,:])[0,:,:,0])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)